# Introduction to Julia by Dr. Mohammad Masiur Rahaman (masiurr@iitbbs.ac.in)

## Probelm No. 1: Generate triangular mesh for a rectangle of length L and height H using Gmsh package in Julia

In [1]:
# input parameters
L = 2;
H = 1;
h = 0.25;

In [ ]:
import Gmsh: gmsh

# Initializing Gmsh (Initialize the Gmsh API)
gmsh.initialize()
# Create a Model (For example, let's create a rectangle of length L and height H)
gmsh.model.add("Rectangle")

# Define points
p1 = gmsh.model.geo.addPoint(0, 0, 0, h)
p2 = gmsh.model.geo.addPoint(L, 0, 0, h)
p3 = gmsh.model.geo.addPoint(L, H, 0, h)
p4 = gmsh.model.geo.addPoint(0, H, 0, h)

# Add lines
l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p1)

# Create a curve loop and surface
cl = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4])
s = gmsh.model.geo.addPlaneSurface([cl])

# Create physical group for domain
gmsh.model.addPhysicalGroup(2, [s], 10, "Domain")

# Create physical groups for boundaries
gmsh.model.addPhysicalGroup(1, [l1], 1, "Bottom")
gmsh.model.addPhysicalGroup(1, [l2], 2, "Right")
gmsh.model.addPhysicalGroup(1, [l3], 3, "Top")
gmsh.model.addPhysicalGroup(1, [l4], 4, "Left")

# Synchronize and Mesh (Synchronize the CAD model and generate a mesh:)
gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)

# Write mesh to file
gmsh.write("rectangle.msh")

#Launch GUI or Finalize (Launch the GUI to visualize the mesh or finalize the Gmsh API:)
gmsh.fltk.run()  # Launch GUI
# or 
gmsh.finalize() # Finalize Gmsh API

### To see the mesh file in paraview 

In [ ]:
using Gridap
using GridapGmsh
model = DiscreteModelFromFile("rectangle.msh")

In [ ]:
writevtk(model,"model")

## Probelm No. 2: Generate rectangular mesh for a rectangle of length L and height H using GridapGmsh package in Julia

In [ ]:
using Gridap
using GridapGmsh

domain = (0,L,0,H)
partition = (10,5)
model = CartesianDiscreteModel(domain,partition)

#Define new boundaries
labels = get_face_labeling(model)
add_tag_from_tags!(labels,"Left",[1,3,7]);
add_tag_from_tags!(labels,"Right",[2,4,8]);
add_tag_from_tags!(labels,"Top",[3,4,6]);
add_tag_from_tags!(labels,"Bottom",[1,2,5]);

In [ ]:
writevtk(model,"modelRectEle")

## Probelm No. 3: Generate triangular mesh for a Brick

In [ ]:
# Define brick dimensions
L, W, H = 2.0, 1.0, 0.5;
h = 0.1;

In [ ]:
using Gmsh

# Initialize Gmsh
gmsh.initialize()

# Create a new model
gmsh.model.add("brick")

# Define corner points
p1 = gmsh.model.geo.addPoint(0, 0, 0, h)
p2 = gmsh.model.geo.addPoint(L, 0, 0, h)
p3 = gmsh.model.geo.addPoint(L, W, 0, h)
p4 = gmsh.model.geo.addPoint(0, W, 0, h)

# Bottom face lines
l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p1)

# Curve loop and plane surface for bottom
cl = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4])
s = gmsh.model.geo.addPlaneSurface([cl])

# Extrude to create 3D brick
ext = gmsh.model.geo.extrude([(p2, s)], 0, 0, H)

# Synchronize geometry
gmsh.model.geo.synchronize()

# Generate 3D mesh
gmsh.model.mesh.generate(3)

# Save mesh
gmsh.write("brick.msh")

#Launch GUI or Finalize (Launch the GUI to visualize the mesh or finalize the Gmsh API:)
gmsh.fltk.run()  # Launch GUI
# Finalize Gmsh
gmsh.finalize()

In [ ]:
using Gridap
using GridapGmsh
model = DiscreteModelFromFile("brick.msh")

writevtk(model,"model3Dbrick")